where varz come from:
    alpha/bezier: random for each phase
    T/time space: random for each phase

    d/motion selector: updated after each phase/k
    t/progress: updated during each phase/k
    q, q/ joints, orient: calculated at each step/k
    a/action: updated at each step/k

test:
    get_robot_joint_positions

methods needed:
    -action
    -step
        -timeout when T is reached
    -reward
done:
    -print obs
    -update obs infra in init
    -reset
    -_get_obs

    -random bezier parameters
    -random time span
    -update motion selector
    -update current and past states
    -update past 6 actions
    -time progress

In [47]:
import numpy as np
import gymnasium
from gymnasium import spaces
from gymnasium.envs.mujoco import MujocoEnv  # Make sure Mujoco is properly installed and imported
from gymnasium import utils
from gymnasium.envs.mujoco import A1SoccerEnv
import random

from scipy.spatial.transform import Rotation as R

In [48]:
# #retired
# class CustomA1SoccerEnv(A1SoccerEnv):
#     def __init__(self):
#         super(CustomA1SoccerEnv, self).__init__()

#         #running the step at 30Hz
#         #0.002 is the time step of the mujoco simulation
#         self.step_dt = 30
#         self.frame_skip = int(((1 / self.step_dt) / 0.002))

#         #1. motion selector, delta
#         self.motion_phase_selector = 0
#         #2. bezier parameters, alpha
#         self.bezier_parameters = np.zeros((3, 5))

#         #3. duration of current motion phase, T_d
#         self.motion_phase_time_span = 0
#         #4. current motion phase progress, t
#         self.motion_phase_progress = 0

#         #5.
#         #robot current state
#         self.robot_joint_positions_current = np.zeros((1, 12))
#         self.robot_orientation_current = np.zeros((1, 3))

#         #robot current and last 6 states
#         self.robot_joint_positions_history = np.zeros((7, 12))
#         self.robot_orientation_history = np.zeros((7, 3))

#         #6. last 6 actions history
#         self.action_current = np.zeros((1, 12))        
#         self.action_history = np.zeros((6, 12))

#     def get_robot_orientation(self):
#         qpos = self.data.qpos
#         # Extract the robot's root orientation quaternion
#         robot_orientation_quat = qpos[10:14]

#         # Convert quaternion to Euler angles
#         # The 'xyz' sequence implies intrinsic rotations around X, Y, Z axes respectively
#         robot_orientation_euler = R.from_quat(robot_orientation_quat).as_euler('xyz', degrees=False)

#         # robot_orientation_euler is an array [roll, pitch, yaw]
#         roll_angle = robot_orientation_euler[0]  # Roll angle about X axis (qψ)
#         pitch_angle = robot_orientation_euler[1] # Pitch angle about Y axis (qθ)
#         yaw_angle = robot_orientation_euler[2]   # Yaw angle about Z axis (qφ)

#         return robot_orientation_euler

#     def update_robot_state(self):
#         qpos = self.data.qpos

#         #joints state update
#         self.robot_joint_positions_current = qpos[14:26]
#         #moving the old current to the back
#         self.robot_joint_positions_history = np.roll(self.robot_joint_positions_history, -1, axis = 0)
#         #replacing the old current with the new current
#         self.robot_joint_positions_history[-1] = self.robot_joint_positions_current

#         #orientation state update
#         self.robot_orientation_current = self.get_robot_orientation()
#         self.robot_orientation_history = np.roll(self.robot_orientation_history, -1, axis = 0)
#         self.robot_orientation_history[-1] = self.robot_orientation_current

#         return self.robot_joint_positions_history, self.robot_orientation_history
    
#     def update_action_history(self):
#         #this method should be called after the action is taken and before a new action is generated
#         self.action_history = np.roll(self.action_history, -1, axis = 0)
#         self.action_history[-1] = self.action_current

#         return self.action_history

#     def update_motion_phase_selector(self):
#         if self.motion_phase_progress == 1:
#             if self.motion_phase_selector == 3:
#                 self.motion_phase_selector = 0
#             else:
#                 self.motion_phase_selector += 1
    
#     def update_motion_phase_progress(self):
#         self.motion_phase_progress += self.step_dt / self.motion_phase_time_span

#         if self.motion_phase_progress > 1:
#             print("Motion phase progress overflow")

#     def initialize_random_bezier(self):
#         #size of each bezier parameter vector
#         size = (3, 1)

#         #an educated guess of bezier parameters range between -0.5m and 0.5m
#         lower_bound = -0.5
#         upper_bound = 0.5

#         #the first random value is for variation, the second is for the initial value
#         #the second value will be substituted by the input from the planning policy once ready
#         alpha_0 = np.random.uniform(-0.1, 0.1, size) + np.random.uniform(lower_bound, upper_bound, size)
#         alpha_1 = np.random.uniform(-0.1, 0.1, size) + np.random.uniform(lower_bound, upper_bound, size)
#         alpha_4 = np.random.uniform(-0.1, 0.1, size) + np.random.uniform(lower_bound, upper_bound, size)

#         alpha_2 = np.random.uniform(-0.1, 0.3, size) + np.random.uniform(lower_bound, upper_bound, size)
#         alpha_3 = np.random.uniform(-0.1, 0.3, size) + np.random.uniform(lower_bound, upper_bound, size)

#         return np.array([alpha_0, alpha_1, alpha_2, alpha_3, alpha_4])

    
#     def initialize_random_time_span(self):
#         #deciding the motion phase time span depending on the kind of motion performed
#         if self.motion_phase_selector == 0:
#             self.motion_phase_time_span = random.uniform(1.0, 4.0)
#         elif self.motion_phase_selector == 1:
#             self.motion_phase_time_span = random.uniform(3.0, 4.0)
#         elif self.motion_phase_time_span == 2:
#             self.motion_phase_time_span = random.uniform(0.2, 0.4)
#         elif self.motion_phase_time_span == 3:
#             self.motion_phase_time_span = random.uniform(1.0, 3.0)

#         return self.motion_phase_time_span

In [49]:
class A1SoccerEnv_v2(A1SoccerEnv):
    def __init__(self):
        super(A1SoccerEnv_v2, self).__init__()

        #running the step at 30Hz
        #0.002 is the time step of the mujoco simulation
        self.step_dt = 30
        self.frame_skip = int(((1 / self.step_dt) / 0.002))

        #1. motion selector, delta
        self.motion_phase_selector = 0
        #2. bezier parameters, alpha
        self.bezier_parameters = np.zeros((3, 5))

        #3. duration of current motion phase, T_d
        self.motion_phase_time_span = 0
        #4. current motion phase progress, t
        self.motion_phase_progress = 0

        #5.
        #robot current state
        self.robot_joint_positions_current = np.zeros((1, 12))
        self.robot_orientation_current = np.zeros((1, 3))

        #robot current and last 6 states
        self.robot_joint_positions_history = np.zeros((7, 12))
        self.robot_orientation_history = np.zeros((7, 3))

        #6. last 6 actions history
        self.action_current = np.zeros((1, 12))        
        self.action_history = np.zeros((6, 12))

    def get_robot_joint_positions(self):
        qpos = self.data.qpos
        joint_positions = qpos[14:26]

        return joint_positions

    def get_robot_orientation(self):
        qpos = self.data.qpos
        # Extract the robot's root orientation quaternion
        robot_orientation_quat = qpos[10:14]

        # Convert quaternion to Euler angles
        # The 'xyz' sequence implies intrinsic rotations around X, Y, Z axes respectively
        robot_orientation_euler = R.from_quat(robot_orientation_quat).as_euler('xyz', degrees=False)

        # robot_orientation_euler is an array [roll, pitch, yaw]
        roll_angle = robot_orientation_euler[0]  # Roll angle about X axis (qψ)
        pitch_angle = robot_orientation_euler[1] # Pitch angle about Y axis (qθ)
        yaw_angle = robot_orientation_euler[2]   # Yaw angle about Z axis (qφ)

        return robot_orientation_euler

    def update_robot_state(self):
        #joints state update
        self.robot_joint_positions_current = self.get_robot_joint_positions()
        #moving the old current to the back
        self.robot_joint_positions_history = np.roll(self.robot_joint_positions_history, -1, axis = 0)
        #replacing the old current with the new current
        self.robot_joint_positions_history[-1] = self.robot_joint_positions_current

        #orientation state update
        self.robot_orientation_current = self.get_robot_orientation()
        self.robot_orientation_history = np.roll(self.robot_orientation_history, -1, axis = 0)
        self.robot_orientation_history[-1] = self.robot_orientation_current

        return self.robot_joint_positions_history, self.robot_orientation_history
    
    def update_action_history(self):
        #this method should be called after the action is taken and before a new action is generated
        self.action_history = np.roll(self.action_history, -1, axis = 0)
        self.action_history[-1] = self.action_current

        return self.action_history

    def update_motion_phase_selector(self):
        if self.motion_phase_progress == 1:
            if self.motion_phase_selector == 3:
                self.motion_phase_selector = 0
            else:
                self.motion_phase_selector += 1
    
    def update_motion_phase_progress(self):
        self.motion_phase_progress += self.step_dt / self.motion_phase_time_span

        if self.motion_phase_progress > 1:
            print("Motion phase progress overflow")

    def initialize_random_bezier(self):
        #size of each bezier parameter vector
        size = (3, 1)

        #an educated guess of bezier parameters range between -0.5m and 0.5m
        lower_bound = -0.5
        upper_bound = 0.5

        #the first random value is for variation, the second is for the initial value
        #the second value will be substituted by the input from the planning policy once ready
        alpha_0 = np.random.uniform(-0.1, 0.1, size) + np.random.uniform(lower_bound, upper_bound, size)
        alpha_1 = np.random.uniform(-0.1, 0.1, size) + np.random.uniform(lower_bound, upper_bound, size)
        alpha_4 = np.random.uniform(-0.1, 0.1, size) + np.random.uniform(lower_bound, upper_bound, size)

        alpha_2 = np.random.uniform(-0.1, 0.3, size) + np.random.uniform(lower_bound, upper_bound, size)
        alpha_3 = np.random.uniform(-0.1, 0.3, size) + np.random.uniform(lower_bound, upper_bound, size)

        return np.array([alpha_0, alpha_1, alpha_2, alpha_3, alpha_4])

    def initialize_random_time_span(self):
        #deciding the motion phase time span depending on the kind of motion performed
        if self.motion_phase_selector == 0:
            self.motion_phase_time_span = random.uniform(1.0, 4.0)
        elif self.motion_phase_selector == 1:
            self.motion_phase_time_span = random.uniform(3.0, 4.0)
        elif self.motion_phase_time_span == 2:
            self.motion_phase_time_span = random.uniform(0.2, 0.4)
        elif self.motion_phase_time_span == 3:
            self.motion_phase_time_span = random.uniform(1.0, 3.0)

        return self.motion_phase_time_span   

    def _get_obs(self):
        qpos = self.data.qpos
        qvel = self.data.qvel

        # Standard observations from MuJoCo
        # saved for later for planning policy
        standard_obs = np.concatenate([
            qpos[:7],      # ball qpos
            qvel[:6],      # ball qvel
            qpos[7:14],    # robot root qpos
            qvel[6:12],    # robot root qvel
            qpos[14:26],   # robot joints qpos
            qvel[12:24]    # robot joints qvel
        ])

        # Combine joint positions and orientations history into one variable
        combined_robot_state_history = np.concatenate([self.robot_joint_positions_history,
                                                        self.robot_orientation_history], axis=1).flatten()

        #Control policy state/observation space
        control_policy_observation = np.concatenate([
            np.array([self.motion_phase_selector]),                # motion selector
                        self.bezier_parameters.flatten(),          # Bézier parameters
                        np.array([self.motion_phase_time_span,     # duration of current motion phase
                        self.motion_phase_progress]),              # current motion phase progress
                        combined_robot_state_history,              # combined history of joint positions and orientations
                        self.action_history.flatten()])            # last 6 actions history

        return control_policy_observation

    def place_components(self):
        #places the robot and goal in random positions, while making sure the robot is facing forward
        noise_low = -self._reset_noise_scale
        noise_high = self._reset_noise_scale

        # Initial positions and velocities
        qpos = np.array(self.init_qpos)
        qvel = np.array(self.init_qvel)

        # Modify robot's orientation to face the goal
        robot_root_orientation_indices = slice(10, 14)

        theta = math.radians(90)
        qw = math.cos(theta / 2)
        qx = 0
        qy = 0
        qz = math.sin(theta / 2)

        robot_orientation = [qw, qx, qy, qz]
        qpos[robot_root_orientation_indices] = robot_orientation

        # Introduce randomness to the ball's initial position
        ball_offset = np.random.uniform(-0.05, 0.05, size=(3,))
        qpos[:3] += ball_offset  # Assuming ball's qpos is first

        # Optional: Introduce some randomness to the robot's root position and orientation for more variability
        qpos[7:10] += self.np_random.uniform(low=noise_low, high=noise_high, size=3)  # Robot root position
        qpos[robot_root_orientation_indices] += self.np_random.uniform(low=-0.05, high=0.05, size=4)  # Robot root orientation

        # Optional: Introduce randomness to the robot's joint positions and velocities
        joint_noise = self.np_random.uniform(low=noise_low, high=noise_high, size=12)  # Assuming 12 joints
        qpos[14:26] += joint_noise

        # Adjusting this line to target the correct slice for robot's joint velocities:
        qvel[12:24] += joint_noise * 0.1  # Smaller noise for velocities


        # Set the state
        self.set_state(qpos, qvel)

    def reset_model(self):
        #places the robot and on the field and initializes its effective joint and orientation values
        self.place_components()

        # Reinitialize custom state components
        self.motion_phase_selector = 0
        self.bezier_parameters = self.initialize_random_bezier()
        self.motion_phase_time_span = self.initialize_random_time_span()
        self.motion_phase_progress = 0

        self.robot_joint_positions_current = np.zeros((1, 12))
        self.robot_joint_positions_history = np.zeros((7, 12))
        self.robot_orientation_current = np.zeros((1, 3))
        self.robot_orientation_history = np.zeros((7, 3))

        self.action_current = np.zeros((1, 12))
        self.action_history = np.zeros((6, 12))

        # Return the observation
        return self._get_obs()

In [51]:
a0 = A1SoccerEnv_v2()

# a0.action_current = np.ones((1, 12))
# a0.update_action_history()
# a0.initialize_random_bezier()

array([[[ 0.23266929],
        [-0.51967158],
        [-0.03861928]],

       [[ 0.27179415],
        [ 0.57185245],
        [-0.13202918]],

       [[-0.32939891],
        [ 0.28534145],
        [ 0.17925581]],

       [[-0.32653504],
        [-0.10983775],
        [ 0.26444323]],

       [[-0.23805258],
        [-0.26037587],
        [-0.10221943]]])

In [2]:
# import gymnasium as gym
# import time

# env = gym.make('A1Soccer', render_mode = 'human')
# env.reset()


# # env.print_obs()
# # while True:
# #     env.render()
# #     # print("OBSERVATION COMING UP")
# #     # print("OBSERVATION: ", env._get_obs())


# while True:  # run for 100 steps or adjust as needed
#     env.step(env.action_space.sample())

#     time.sleep(0.25)
#     print("OBSERVATION COMING UP")

#     print(env.update_robot_state())
#     #env.print_obs()
#       # you can use a random action or some default action
#     env.render()